In [4]:
!pip install PyPDF2 tqdm

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)


In [5]:
from PyPDF2 import PdfReader
from tqdm import tqdm
reader = PdfReader("Data/Psychology2e_WEB.pdf")

text = ""
for page in tqdm(reader.pages):
    text += page.extract_text() + "\n"

with open("Data/Psychology2e_WEB.txt", "w", encoding="utf-8") as f:
    f.write(text)
print("PDF text extraction complete. Text saved to 'Data/Psychology2e_WEB.txt'.")

100%|██████████| 753/753 [00:17<00:00, 43.31it/s]

PDF text extraction complete. Text saved to 'Data/Psychology2e_WEB.txt'.


In [8]:
!pip install pdfminer.six


  Using cached charset_normalizer-3.4.2-cp311-cp311-win_amd64.whl.metadata (36 kB)
  Using cached cffi-1.17.1-cp311-cp311-win_amd64.whl.metadata (1.6 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
   ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
   ---------------------------------------- 5.6/5.6 MB 34.3 MB/s eta 0:00:00
Using cached charset_normalizer-3.4.2-cp311-cp311-win_amd64.whl (105 kB)
   ---------------------------------------- 0.0/3.4 MB ? eta -:--:--
   ---------------------------------------- 3.4/3.4 MB 67.0 MB/s eta 0:00:00
Using cached cffi-1.17.1-cp311-cp311-win_amd64.whl (181 kB)
Using cached pycparser-2.22-py3-none-any.whl (117 kB)

   ---------------------------------------- 0/5 [pycparser]
   ---------------------------------------- 0/5 [pycparser]
   ---------------------------------------- 0/5 [pycparser]
   -------- ------------------------------- 1/5 [charset-normalizer]
   -------- ------------------------------- 1/5 [

In [9]:
from pdfminer.high_level import extract_text
from tqdm import tqdm
import os

pdf_path = "Data/Psychology2e_WEB.pdf"
txt_path = "Data/Psychology2e_WEB.txt"

# Extract all text from PDF
print("Extracting text with pdfminer...")
text = extract_text(pdf_path)

# Save to .txt file
os.makedirs(os.path.dirname(txt_path), exist_ok=True)
with open(txt_path, "w", encoding="utf-8") as f:
    f.write(text)

print(f"PDF text extraction complete. Text saved to '{txt_path}'.")


Extracting text with pdfminer...
PDF text extraction complete. Text saved to 'Data/Psychology2e_WEB.txt'.


In [10]:
import difflib


In [11]:
from PyPDF2 import PdfReader
from pdfminer.high_level import extract_text
from tqdm import tqdm
import difflib
import os

# ----------- Step 1: Extract with PyPDF2 -----------
print("Extracting text with PyPDF2...")
reader = PdfReader("Data/Psychology2e_WEB.pdf")
text1 = ""
for page in tqdm(reader.pages):
    text1 += page.extract_text() + "\n"

# Save PyPDF2 output
with open("Data/psych_text_pypdf2.txt", "w", encoding="utf-8") as f:
    f.write(text1)
print("Saved PyPDF2 output to psych_text_pypdf2.txt")

# ----------- Step 2: Extract with pdfminer -----------
print("Extracting text with pdfminer...")
text2 = extract_text("Data/Psychology2e_WEB.pdf")

# Save pdfminer output
with open("Data/psych_text_pdfminer.txt", "w", encoding="utf-8") as f:
    f.write(text2)
print("Saved pdfminer output to psych_text_pdfminer.txt")

# ----------- Step 3: Compare Texts -----------
print("Comparing extracted texts...")
diff = difflib.unified_diff(
    text1.splitlines(),
    text2.splitlines(),
    fromfile='PyPDF2',
    tofile='pdfminer',
    lineterm=''
)

with open("text_diff.txt", "w", encoding="utf-8") as f:
    for line in diff:
        f.write(line + "\n")

print("Comparison saved to text_diff.txt")


Extracting text with PyPDF2...


100%|██████████| 753/753 [00:17<00:00, 44.11it/s]


Saved PyPDF2 output to psych_text_pypdf2.txt
Extracting text with pdfminer...
Saved pdfminer output to psych_text_pdfminer.txt
Comparing extracted texts...
Comparison saved to text_diff.txt


In [1]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
from tqdm import tqdm

# high_level import extract_text

# Specify the path to your PDF file
pdf_path = "Data/Psychology2e_WEB.pdf"

rsrc_mgr = PDFResourceManager()
retstr = StringIO()
laparams = LAParams()
device = TextConverter(rsrc_mgr, retstr, laparams=laparams)
interpreter = PDFPageInterpreter(rsrc_mgr, device)

# Open the PDF file
with open(pdf_path, "rb") as fp:
    # Process each page in the PDF

    # The first 18 pages are not needed, so we will skip them
    # References are also not needed. So we will skip these as well
    page_number = 0
    for page in tqdm(PDFPage.get_pages(fp)):
        page_number += 1
        if page_number <= 18 or page_number > 644:
            continue
        interpreter.process_page(page)
    # Get the text from the StringIO object
    text = retstr.getvalue()
# Close the device and StringIO object
device.close()
retstr.close()

# Print or save the extracted text
text = text.replace("\n\nAccess for free at openstax.org", "") 
# Optionally, save the text to a file
with open("Data/Psychology2e_WEB_pdfminer_trimmed.txt", "w", encoding="utf-8") as text_file:
    text_file.write(text)

753it [00:22, 33.40it/s]


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

with open("Data/Psychology2e_WEB_pdfminer_trimmed.txt", "r", encoding="utf-8") as f:
    full_text = f.read()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=50, separators=["\n\n", "\n", ".", " "]
)
chunks = splitter.create_documents([full_text])


In [39]:
len(chunks)

4269

In [40]:
chunks[0]

Document(metadata={}, page_content='Introduction to Psychology\n\n1\n\nFIGURE 1.1 Psychology is the scientific study of mind and behavior. (credit "background": modification of work by\n\nNattachai Noogure; credit "top left": modification of work by Peter Shanks; credit "top middle": modification of work\n\nby "devinf"/Flickr; credit "top right": modification of work by Alejandra Quintero Sinisterra; credit "bottom left":\n\nmodification of work by Gabriel Rocha; credit "bottom middle-left": modification of work by Caleb Roenigk; credit')

In [42]:
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer("all-MiniLM-L6-v2") 

In [51]:
from tqdm import tqdm
import pinecone
from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings

In [44]:
import os
pinecone_api_key = "pcsk_4cC2jC_DYQseHxA8jge2oFn6o7SV5F6aMYJqWXuQzUTJpqzhwyiHLyahNYqvobky7emeoT"

In [53]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=pinecone_api_key)

index_name = "casml-py"
# Initialize embedding model
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,  
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

In [54]:
# Step 1: Connect to Pinecone
index = pc.Index(index_name)

batch_size = 100
for i in tqdm(range(0, len(chunks), batch_size), desc="Uploading batches"):
    batch = chunks[i:i + batch_size]
    texts = [doc.page_content for doc in batch]
    ids = [f"chunk-{i+j}" for j in range(len(batch))]
    vectors = embedder.encode(texts).tolist()

    to_upsert = [
        {"id": ids[j], "values": vectors[j], "metadata": {"text": texts[j]}}
        for j in range(len(batch))
    ]
    index.upsert(vectors=to_upsert)

print(f"✅ Successfully upserted {len(chunks)} chunks to Pinecone index '{index_name}'.")

Uploading batches: 100%|██████████| 43/43 [00:50<00:00,  1.18s/it]

✅ Successfully upserted 4269 chunks to Pinecone index 'casml-py'.


In [55]:
query = "What is psychology?"
query_vector = embedder.encode([query]).tolist()

# Search top 5 similar chunks
results = index.query(vector=query_vector[0], top_k=5, include_metadata=True)

for match in results['matches']:
    print(f"\n📌 Score: {match['score']}")
    print(match['metadata']['text'])



📌 Score: 0.736447394
psychology is the area of psychology that focuses on studying cognitions, or thoughts, and their relationship to

our experiences and our actions. Like biological psychology, cognitive psychology is broad in its scope and

often involves collaborations among people from a diverse range of disciplinary backgrounds. This has led

some to coin the term cognitive science to describe the interdisciplinary nature of this area of research (Miller,

2003).

📌 Score: 0.720964491
explores questions like these. Psychology refers to the scientific study of the mind and behavior. Psychologists

use the scientific method to acquire knowledge. To apply the scientific method, a researcher with a question

about how or why something happens will propose a tentative explanation, called a hypothesis, to explain the

phenomenon. A hypothesis should fit into the context of a scientific theory, which is a broad explanation or

📌 Score: 0.649055541
disorders and other problematic patter

In [5]:
from pdfminer.high_level import extract_text
from pdfminer.pdfpage import PDFPage
from tqdm import tqdm

pdf_path = "Data/Psychology2e_WEB.pdf"
page_texts = []

with open(pdf_path, "rb") as f:
    for i, page in enumerate(tqdm(PDFPage.get_pages(f), desc="Extracting pages"), 1):
        if i <= 18 or i > 644:
            continue
        text = extract_text(pdf_path, page_numbers=[i - 1])  # 0-based
        text = text.replace("\n\nAccess for free at openstax.org", "")
        page_texts.append((i, text))


Extracting pages: 369it [01:56,  3.16it/s]


KeyboardInterrupt: 

1

In [16]:
from pdfminer.high_level import extract_text
from pdfminer.pdfpage import PDFPage
from tqdm import tqdm
import re

pdf_path = "Data/Psychology2e_WEB.pdf"
page_texts = []

with open(pdf_path, "rb") as f:
    for i, page in enumerate(tqdm(PDFPage.get_pages(f), desc="Extracting pages"), 1):
        text = extract_text(pdf_path, page_numbers=[i - 1])
        text = text.replace("\n\nAccess for free at openstax.org", "")
        page_texts.append((i, text))


def convert_to_markdown_headers(text):
    text = re.sub(r"\bCHAPTER\s+(\d+)\s+(.*)", r"# \1 \2", text, flags=re.IGNORECASE)
    text = re.sub(r"\n(\d{1,2}\.\d+)\s+(.*)", r"\n## \1 \2", text)
    return text

markdown_pages = [(p, convert_to_markdown_headers(t)) for p, t in page_texts]        

Extracting pages: 753it [03:52,  3.24it/s]


2

In [31]:
from langchain.text_splitter import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter

header_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[("#", "chapter"), ("##", "section")])
docs_with_meta = []

for page_num, md_text in markdown_pages:
    page_docs = header_splitter.split_text(md_text)
    for d in page_docs:
        d.metadata["page"] = page_num
    docs_with_meta.extend(page_docs)

chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

final_chunks = []
for doc in docs_with_meta:
    chunks = chunk_splitter.create_documents([doc.page_content])
    for chunk in chunks:
        chunk.metadata.update(doc.metadata)
    final_chunks.extend(chunks)


4

In [32]:
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec

embedder = SentenceTransformer("all-MiniLM-L6-v2")

pinecone_api_key = "pcsk_4cC2jC_DYQseHxA8jge2oFn6o7SV5F6aMYJqWXuQzUTJpqzhwyiHLyahNYqvobky7emeoT"
index_name = "casml-py"
pc = Pinecone(api_key=pinecone_api_key)

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

index = pc.Index(index_name)


In [29]:
# Delete the index
# if pc.has_index(index_name):
#     pc.delete_index(index_name)
#     print(f"Index '{index_name}' has been deleted.")
# else:
#     print(f"Index '{index_name}' does not exist.")

Index 'casml-py' has been deleted.


In [33]:
from tqdm import tqdm

batch_size = 100
for i in tqdm(range(0, len(final_chunks), batch_size), desc="Uploading to Pinecone"):
    batch = final_chunks[i:i + batch_size]
    texts = [doc.page_content for doc in batch]
    ids = [f"chunk-{i+j}" for j in range(len(batch))]
    vectors = embedder.encode(texts).tolist()
    to_upsert = [
        {
            "id": ids[j],
            "values": vectors[j],
            "metadata": {
                **batch[j].metadata,
                "text": texts[j]  # ✅ Now included for query results
            }
        }
        for j in range(len(batch))
    ]
    index.upsert(vectors=to_upsert)


Uploading to Pinecone: 100%|██████████| 57/57 [01:13<00:00,  1.28s/it]


In [1]:
# Your search query
query = "What is cognitive psychology?"
query_vector = embedder.encode([query]).tolist()

# Query Pinecone
results = index.query(vector=query_vector[0], top_k=5, include_metadata=True)

# Display the results
for match in results["matches"]:
    print(f"\nScore: {match['score']:.4f}")
    print(f"Page: {match['metadata'].get('page')}")
    print(f"Chapter: {match['metadata'].get('chapter')}")
    print(f"Section: {match['metadata'].get('section')}")
    print(f"\nText:\n{match['metadata']['text'][:500]}...")


NameError: name 'embedder' is not defined